In [ ]:
!pip install aiohttp nest_asyncio
!pip install pyngrok

In [ ]:
import psutil
import os

def kill_port(port):
    for proc in psutil.process_iter(['pid', 'name']):
        for conn in proc.connections(kind='inet'):
            if conn.laddr.port == port:
                os.kill(proc.info['pid'], 9)  # Beende den Prozess auf dem angegebenen Port
                print(f"Prozess {proc.info['pid']} auf Port {port} wurde beendet.")

# Port 8081 freigeben, falls er belegt ist
kill_port(8081)

In [ ]:
import asyncio
from aiohttp import web
import nest_asyncio
from pyngrok import ngrok
import os
import subprocess
import json
import git  # GitPython

# nest_asyncio anwenden, um mehrfaches Starten des Event-Loops zu ermöglichen
nest_asyncio.apply()

# Überprüfen, ob das Projekt und der spezifische Branch bereits vorhanden sind
def check_project_version(project_path, branch):
    if not os.path.exists(project_path):
        print(f"Projektpfad {project_path} existiert nicht.")
        return False
    
    try:
        repo = git.Repo(project_path)
        # Liste der remote Branches holen und prüfen
        remote_branches = [ref.name for ref in repo.remotes.origin.refs]
        if f'origin/{branch}' in remote_branches:
            print(f"Branch '{branch}' ist bereits lokal vorhanden.")
            return True
        else:
            print(f"Branch '{branch}' ist nicht lokal vorhanden.")
            return False
    except git.exc.InvalidGitRepositoryError:
        print(f"{project_path} ist kein gültiges Git-Repository.")
        return False
    except Exception as e:
        print(f"Fehler bei der Überprüfung des Branches: {str(e)}")
        return False

# Funktion zum Klonen des Projekts von GitHub mit einem spezifischen Branch
def clone_project_from_github(project_name, branch, project_path):
    github_url = f"https://github.com/your_repo/{project_name}.git"
    branch = branch if branch else 'main'  # Verwende 'main', wenn kein Branch übergeben wurde
    try:
        if os.path.exists(project_path):
            print(f"Projektpfad {project_path} existiert bereits. Aktualisiere Repository.")
            repo = git.Repo(project_path)
            repo.git.checkout(branch)
            repo.remotes.origin.pull(branch)
        else:
            if os.path.exists(project_path):
                shutil.rmtree(project_path)  # Lösche den alten Pfad, falls er existiert
            print(f"Klonen des Branches '{branch}' von {github_url}")
            git.Repo.clone_from(github_url, project_path, branch=branch)
        return True, f"Branch '{branch}' erfolgreich geklont oder aktualisiert."
    except Exception as e:
        return False, str(e)

# Funktion zur Ausführung des Skripts mit den Parametern
async def execute_script(project_path, script_name, parameters):
    script_path = os.path.join(project_path, script_name)
    if not os.path.exists(script_path):
        return False, f"Skript '{script_name}' nicht gefunden im Projektpfad '{project_path}'"
    
    # Führe das Skript mit den Parametern aus
    cmd = ['python3', script_path] + parameters
    process = await asyncio.create_subprocess_exec(*cmd, stdout=asyncio.subprocess.PIPE, stderr=asyncio.subprocess.PIPE)
    stdout, stderr = await process.communicate()

    if process.returncode == 0:
        return True, stdout.decode()
    else:
        return False, stderr.decode()

# Request-Handler für Airflow-Aufträge
async def handle_request(request):
    try:
        data = await request.json()
        project_name = data.get('project')
        branch = data.get('branch', 'main')  # Verwende 'main', wenn kein Branch übergeben wurde
        script_name = data.get('script')
        parameters = data.get('parameters', [])

        if not all([project_name, branch, script_name]):
            return web.Response(status=400, text="Fehlende Projektinformationen (Projekt, Branch, Skript).")

        # Überprüfen und Klonen/Aktualisieren des Projekts
        project_path = f"/path/to/projects/{project_name}/{branch}"
        success, message = clone_project_from_github(project_name, branch, project_path)
        if not success:
            return web.Response(status=500, text=f"Fehler beim Klonen/Aktualisieren des Projekts: {message}")
        
        # Ausführen des Skripts und Rückmeldung
        success, output = await execute_script(project_path, script_name, parameters)
        if success:
            return web.Response(text=f"Skript erfolgreich ausgeführt. Ausgabe:\n{output}")
        else:
            return web.Response(status=500, text=f"Fehler bei der Skriptausführung: {output}")
    
    except json.JSONDecodeError:
        return web.Response(status=400, text="Ungültiges JSON im Request.")
    except Exception as e:
        return web.Response(status=500, text=f"Interner Fehler: {str(e)}")


# Initialisiere den Webserver
async def init_app():
    app = web.Application()
    app.router.add_post('/trigger', handle_request)
    return app

# Webserver und ngrok starten
async def main():
    app = await init_app()
    runner = web.AppRunner(app)
    await runner.setup()
    site = web.TCPSite(runner, '127.0.0.1', 4041)
    await site.start()
    print("Server läuft auf http://127.0.0.1:4041")

    authtoken = "2ltJwdZ4ItmsDom35mIM3kz7wCT_6yKrmv2HpAMhZemSV2iwa"
    ngrok.kill()
    await run_process(['ngrok', 'config', 'add-authtoken', authtoken])
    await asyncio.gather(
        run_process(['ngrok', 'http', '--log', 'stderr', '4041', '--host-header', 'localhost:4041'])
    )

    while True:
        await asyncio.sleep(3600)

# Prozesse ausführen
async def run_process(cmd):
    p = await asyncio.subprocess.create_subprocess_exec(
        *cmd,
        stdout=asyncio.subprocess.PIPE,
        stderr=asyncio.subprocess.PIPE,
    )

    async def pipe(lines):
        async for line in lines:
            print(line.strip().decode('utf-8'))

    await asyncio.gather(pipe(p.stdout), pipe(p.stderr))

# Event-Loop starten
loop = asyncio.get_event_loop()
loop.run_until_complete(main())
